In [1]:
from sympy import symbols, IndexedBase, Idx
from sympy.tensor.index_methods import get_indices
#from sympy.abc import *
u = IndexedBase('u', shape=(3,))
i, j, k = symbols('i j k', cls=Idx)
#i, j, k = symbols('i j k', integer=True)
uiuj = u[i]*u[j]
print(get_indices(uiuj))

({j, i}, {})


In [13]:
from sympy.tensor.tensor import TensorIndexType, tensor_indices, TensorHead, TensorSymmetry, TensorManager, \
TensAdd, TensMul
from sympy.tensor.toperators import PartialDerivative
Ind = TensorIndexType('Index')#, dummy_name='I')
u
i, j, k, l = tensor_indices('i, j, k, l', Ind)
x, y, z, t = symbols('x, y, z, t')
u_x, u_y, u_z = symbols('u_x, u_y, u_z')
u = TensorHead('u', [Ind], TensorSymmetry.fully_symmetric(1), 'ucomm')
rho = TensorHead('rho', [], TensorSymmetry.fully_symmetric(0), 'ucomm')
TensorManager.set_comm('ucomm', 'ucomm', 0)
d = TensorHead('d', [Ind], TensorSymmetry.fully_symmetric(1), 'dcomm')
dt = TensorHead('dt', [], TensorSymmetry.fully_symmetric(0), 'dcomm')
TensorManager.set_comm('dcomm', 'dcomm', 0)

A = u(i)*u(j)-u(j)*u(i)
B = u(j)*u(i)
C = dt()*d(-i)*B
D = d(-i)*dt()*B
E = dt()*dt()*u(-j)
print(dt.name)

dt


In [14]:
print(C*E)

dt*d(-I_1)*u(I_0)*u(I_1)*dt*dt*u(-I_0)


In [15]:
du = PartialDerivative(u(i), t)
print(du._expand_partial_derivative())
print(du.canon_bp())

PartialDerivative(u(i), t)


AttributeError: 'PartialDerivative' object has no attribute 'canon_bp'

In [16]:
class PD(PartialDerivative):
    def canon_bp(self):
        expr = self._expand_partial_derivative()
        if isinstance(expr, TensAdd):
            return expr.canon_bp()
        if not expr.components:
            return expr
        t = expr.sorted_components()
        g, dummies, msym = t._index_structure.indices_canon_args()
        v = components_canon_args(t.components)
        can = canonicalize(g, dummies, msym, *v)
        if can == 0:
            return S.Zero
        tmul = t.perm2tensor(can, True)
        return tmul

In [22]:
du = PD(u(i),t)
print(PD.expr)